# Simple Model Example

There are two ways of running a simple eCalc™ model:

1. Specifying a YAML model
2. Specifying a Python model

Here we will look at both scenarios:

## The Simple Model
The Simple used in this example is the reference case in [eCalc™ Docs - Simple Model Example](https://equinor.github.io/ecalc/docs/about/modelling/examples/simple).

The model consists of a single installation with:
- Flare
- Gas Export Compressor
- Generator set A:
    - Base Production Load
    - Gas Injection Compressor
    - Produced Water Re-injection Pump
    - Seawater Injection Pump

The model.yaml file contains references to:
- production_data.csv
- genset.csv
- compressor_sampled.csv
- compressor_sampled_with_turbine.csv
- pump_sampled.csv
- pump_chart.csv

### Python Model
We will now recreate the exact same model, but this time using the libecalc Python library

We start by defining some variables that we will use later

VARIABLES:
- hydrocarbon_export_sm3_per_day
- salt_water_injection_rate_m3_per_day
- gas_export_rate_sm3_per_day
- gas_injection_rate_sm3_per_day
- produced_water_reinjection_condition
- produced_water_reinjection_total_system_rate_m3_per_day
- flare_fuel_rate_sm3_day


In [ ]:
from libecalc.dto.types import ConsumptionType, EnergyUsageType, ConsumerType, EnergyModelType
from libecalc.common.utils.rates import RateType
from libecalc.dto.base import ConsumerUserDefinedCategoryType, ComponentType
from libecalc.expression import Expression
from libecalc.core.graph_result import GraphResult
from libecalc.core.ecalc import EnergyCalculator
from datetime import datetime
from libecalc.dto import (
    DirectConsumerFunction,
    Asset,
    Installation,
    GeneratorSet,
    GeneratorSetSampled,
    ElectricityConsumer,
    PumpConsumerFunction,
    PumpModel,
    SingleSpeedChart,
    TabulatedConsumerFunction,
    TabulatedData,
    Variables,
    CompressorSampled,
    CompressorConsumerFunction,
    FuelConsumer,
    Emission,
    FuelType,
    VariablesMap
)

In [ ]:
variables = VariablesMap(
    time_vector=[
        datetime(2020, 1, 1, 0, 0),
        datetime(2021, 1, 1, 0, 0),
        datetime(2022, 1, 1, 0, 0),
        datetime(2023, 1, 1, 0, 0),
        datetime(2024, 1, 1, 0, 0),
        datetime(2024, 12, 1, 0, 0),
        datetime(2026, 1, 1, 0, 0),
        datetime(2027, 1, 1, 0, 0),
        datetime(2028, 1, 1, 0, 0),
        datetime(2029, 1, 1, 0, 0),
        datetime(2030, 1, 1, 0, 0),
        datetime(2031, 1, 1, 0, 0),
    ],
    variables={
        "SIM;OIL_PROD": [
            9000.0,
            8000.0,
            7000.0,
            6000.0,
            6000.0,
            6000.0,
            7000.0,
            6000.0,
            6000.0,
            5000.0,
            6000.0,
            4000.0,
        ],
        "SIM;GAS_SALES": [
            3500000.0,
            3600000.0,
            3700000.0,
            3800000.0,
            3900000.0,
            4000000.0,
            4100000.0,
            4500000.0,
            3500000.0,
            2500000.0,
            2000000.0,
            3000000.0,
        ],
        "SIM;WATER_PROD": [
            18000.0,
            19000.0,
            15000.0,
            16000.0,
            14000.0,
            15000.0,
            18000.0,
            15000.0,
            12000.0,
            14000.0,
            16000.0,
            14000.0,
        ],
        "SIM;WATER_INJ": [
            34000.0,
            33000.0,
            30000.0,
            33000.0,
            35000.0,
            36000.0,
            36000.0,
            38000.0,
            33000.0,
            36000.0,
            35000.0,
            33000.0,
        ],
        "SIM;GAS_INJ": [
            220000000.0,
            220000000.0,
            220000000.0,
            230000000.0,
            230000000.0,
            240000000.0,
            240000000.0,
            240000000.0,
            240000000.0,
            240000000.0,
            240000000.0,
            240000000.0,
        ],
        "SIM;GAS_LIFT": [
            13000000.0,
            17000000.0,
            21000000.0,
            24000000.0,
            28000000.0,
            31000000.0,
            35000000.0,
            39000000.0,
            43000000.0,
            46000000.0,
            50000000.0,
            53000000.0,
        ],
        "$var.hydrocarbon_export_sm3_per_day": [
            12500.0,
            11600.0,
            10700.0,
            9800.0,
            9900.0,
            10000.0,
            11100.0,
            10500.0,
            9500.0,
            7500.0,
            8000.0,
            7000.0,
        ],
        "$var.gas_export_rate_sm3_per_day": [
            3500000.0,
            3600000.0,
            3700000.0,
            3800000.0,
            3900000.0,
            4000000.0,
            4100000.0,
            4500000.0,
            3500000.0,
            2500000.0,
            2000000.0,
            3000000.0,
        ],
        "$var.produced_water_reinjection_condition": [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
        "$var.flare_fuel_rate_sm3_day": [
            7000.0,
            7000.0,
            7000.0,
            7000.0,
            7000.0,
            7000.0,
            7000.0,
            7000.0,
            7000.0,
            7000.0,
            7000.0,
            7000.0,
        ],
        "$var.salt_water_injection_rate_m3_per_day": [
            17000.0,
            14000.0,
            15000.0,
            17000.0,
            21000.0,
            21000.0,
            19000.0,
            23000.0,
            21000.0,
            22000.0,
            19000.0,
            19000.0,
        ],
        "$var.produced_water_reinjection_total_system_rate_m3_per_day": [
            18000.0,
            19000.0,
            15000.0,
            16000.0,
            14000.0,
            15000.0,
            18000.0,
            15000.0,
            12000.0,
            14000.0,
            16000.0,
            14000.0,
        ],
        "$var.gas_injection_rate_sm3_per_day": [
            233000000.0,
            237000000.0,
            241000000.0,
            254000000.0,
            258000000.0,
            271000000.0,
            275000000.0,
            279000000.0,
            283000000.0,
            286000000.0,
            290000000.0,
            293000000.0,
        ],
    },
)


Then we create fuel and emissions that will be used by the model.

In [ ]:
co2 = Emission(
    name="co2",
    factor=Expression.setup_from_expression(2.19),
)

fuel_gas = FuelType(
    name="fuel_gas", user_defined_category=None, emissions=[co2]
)

Next we create the Flare

In [ ]:
flare = FuelConsumer(
    component_type=ComponentType.GENERIC,
    name="Flare",
    regularity={datetime(1900, 1, 1): Expression.setup_from_expression(1.0)},
    user_defined_category={datetime(1900, 1, 1): ConsumerUserDefinedCategoryType.FLARE},
    consumes=ConsumptionType.FUEL,
    fuel={datetime(1900, 1, 1): fuel_gas},
    energy_usage_model={
        datetime(1900, 1, 1): DirectConsumerFunction(
            typ=ConsumerType.DIRECT,
            energy_usage_type=EnergyUsageType.FUEL,
            condition=None,
            fuel_rate=Expression.setup_from_expression("$var.flare_fuel_rate_sm3_day"),
            load=None,
            power_loss_factor=None,
            consumption_rate_type=RateType.STREAM_DAY,
        )
    },
)

Then we create a gas export compressor

In [ ]:
gas_export_compressor = FuelConsumer(
    component_type=ComponentType.COMPRESSOR,
    name="Gas export compressor",
    regularity={datetime(1900, 1, 1): Expression.setup_from_expression(1.0)},
    user_defined_category={datetime(1900, 1, 1): ConsumerUserDefinedCategoryType.COMPRESSOR},
    consumes=ConsumptionType.FUEL,
    fuel={datetime(1900, 1, 1): fuel_gas},
    energy_usage_model={
        datetime(1900, 1, 1): CompressorConsumerFunction(
            typ=ConsumerType.COMPRESSOR,
            energy_usage_type=EnergyUsageType.FUEL,
            condition=None,
            power_loss_factor=None,
            model=CompressorSampled(
                energy_usage_adjustment_constant=0.0,
                energy_usage_adjustment_factor=1.0,
                typ=EnergyModelType.COMPRESSOR_SAMPLED,
                energy_usage_type=EnergyUsageType.FUEL,
                energy_usage_values=[0.0, 50000.0, 50000.0, 130000.0, 170000.0],
                rate_values=[0.0, 0.1, 3000000.0, 3500000.0, 7000000.0],
                suction_pressure_values=None,
                discharge_pressure_values=None,
                power_interpolation_values=None,
            ),
            rate_standard_m3_day=Expression.setup_from_expression("$var.gas_export_rate_sm3_per_day"),
            suction_pressure=None,
            discharge_pressure=None,
            interstage_control_pressure=None,
        )
    },
)

Then we create the base production load

In [ ]:


base_production_load = ElectricityConsumer(
    component_type=ComponentType.GENERIC,
    name="Base production load",
    regularity={datetime(1900, 1, 1): Expression.setup_from_expression(1.0)},
    user_defined_category={datetime(1900, 1, 1): ConsumerUserDefinedCategoryType.BASE_LOAD},
    consumes=ConsumptionType.ELECTRICITY,
    fuel=None,
    energy_usage_model={
        datetime(1900, 1, 1): DirectConsumerFunction(
            energy_usage_type=EnergyUsageType.POWER,
            condition=None,
            fuel_rate=None,
            load=Expression.setup_from_expression(11.8),
            power_loss_factor=None,
            consumption_rate_type=RateType.STREAM_DAY,
        )
    },
)


Then we create a gas injection compressor

In [ ]:
gas_injection_compressor = ElectricityConsumer(
    name="Gas injection compressor",
    regularity={datetime(1900, 1, 1): Expression.setup_from_expression(1.0)},
    user_defined_category={datetime(1900, 1, 1): ConsumerUserDefinedCategoryType.COMPRESSOR},
    consumes=ConsumptionType.ELECTRICITY,
    fuel=None,
    energy_usage_model={
        datetime(1900, 1, 1): CompressorConsumerFunction(
            typ=ConsumerType.COMPRESSOR,
            energy_usage_type=EnergyUsageType.POWER,
            condition=None,
            power_loss_factor=None,
            model=CompressorSampled(
                energy_usage_adjustment_constant=0.0,
                energy_usage_adjustment_factor=1.0,
                typ=EnergyModelType.COMPRESSOR_SAMPLED,
                energy_usage_type=EnergyUsageType.POWER,
                energy_usage_values=[0.0, 4.1, 4.1, 4.1, 4.1, 4.4, 4.8, 5.1, 5.4, 5.8, 6.1, 6.4, 6.8, 7.1, 14.2],
                rate_values=[
                    0.0,
                    1.0,
                    100000000.0,
                    200000000.0,
                    210000000.0,
                    220000000.0,
                    230000000.0,
                    240000000.0,
                    250000000.0,
                    260000000.0,
                    270000000.0,
                    280000000.0,
                    290000000.0,
                    300000000.0,
                    500000000.0,
                ],
                suction_pressure_values=None,
                discharge_pressure_values=None,
                power_interpolation_values=None,
            ),
            rate_standard_m3_day=Expression.setup_from_expression("$var.gas_injection_rate_sm3_per_day"),
            suction_pressure=None,
            discharge_pressure=None,
            interstage_control_pressure=None,
        )
    },
    component_type=ComponentType.COMPRESSOR,
)

Then we create a Produced Water Re-injection Pump

In [ ]:
produced_water_reinjection_pump = ElectricityConsumer(
    component_type=ComponentType.PUMP,
    name="Produced water reinjection pump",
    regularity={datetime(1900, 1, 1): Expression.setup_from_expression(1.0)},
    user_defined_category={
        datetime(1900, 1, 1): ConsumerUserDefinedCategoryType.PUMP,
    },
    consumes=ConsumptionType.ELECTRICITY,
    fuel=None,
    energy_usage_model={
        datetime(1900, 1, 1): PumpConsumerFunction(
            typ=ConsumerType.PUMP,
            energy_usage_type=EnergyUsageType.POWER,
            condition=Expression.setup_from_expression("$var.produced_water_reinjection_condition"),
            power_loss_factor=None,
            model=PumpModel(
                energy_usage_adjustment_constant=0.0,
                energy_usage_adjustment_factor=1.0,
                typ=EnergyModelType.PUMP_MODEL,
                chart=SingleSpeedChart(
                    speed_rpm=3250.0,
                    rate_actual_m3_hour=[250.0, 360.0, 500.0, 600.0, 667.0, 735.0],
                    polytropic_head_joule_per_kg=[25898.4, 24426.9, 22975.02, 21680.1, 20287.08, 18344.7],
                    efficiency_fraction=[0.59, 0.68, 0.77, 0.8, 0.78, 0.74],
                ),
                head_margin=0.0,
            ),
            rate_standard_m3_day=Expression.setup_from_expression(
                "$var.produced_water_reinjection_total_system_rate_m3_per_day"
            ),
            suction_pressure=Expression.setup_from_expression(10.0),
            discharge_pressure=Expression.setup_from_expression(200.0),
            fluid_density=Expression.setup_from_expression(1010.0),
        )
    },
)


The we create a Seawater Injection Pump

In [ ]:
seawater_injection_pump = ElectricityConsumer(
    component_type=ComponentType.GENERIC,
    name="Sea water injection pump",
    regularity={datetime(1900, 1, 1): Expression.setup_from_expression(1.0)},
    user_defined_category={datetime(1900, 1, 1): ConsumerUserDefinedCategoryType.PUMP},
    consumes=ConsumptionType.ELECTRICITY,
    fuel=None,
    energy_usage_model={
        datetime(1900, 1, 1): TabulatedConsumerFunction(
            typ=ConsumerType.TABULATED,
            energy_usage_type=EnergyUsageType.POWER,
            condition=None,
            power_loss_factor=None,
            model=TabulatedData(
                energy_usage_adjustment_constant=0.0,
                energy_usage_adjustment_factor=1.0,
                headers=["RATE", "POWER"],
                data=[[0.0, 1.0, 8500.0, 9000.0, 17000.0, 17500.0, 36000.0], [0.0, 3.0, 4.0, 4.0, 6.0, 9.0, 13.0]],
                typ=EnergyModelType.TABULATED,
            ),
            variables=[
                Variables(
                    name="RATE",
                    expression=Expression.setup_from_expression("$var.salt_water_injection_rate_m3_per_day"),
                )
            ],
        )
    },
)

Then we collect the electricity consumers inside a GeneratorSet

In [ ]:
generator_set_a = GeneratorSet(
    name="Generator set A",
    regularity={datetime(1900, 1, 1): Expression.setup_from_expression(1.0)},
    user_defined_category={datetime(1900, 1, 1): ConsumerUserDefinedCategoryType.TURBINE_GENERATOR},
    fuel={datetime(1900, 1, 1): fuel_gas},
    generator_set_model={
        datetime(1900, 1, 1): GeneratorSetSampled(
            energy_usage_adjustment_constant=0.0,
            energy_usage_adjustment_factor=1.0,
            headers=["POWER", "FUEL"],
            data=[[0.0, 0.1, 10.0, 20.0, 40.0, 100.0], [0.0, 65000.0, 75000.0, 126000.0, 250000.0, 750000.0]],
        )
    },
    consumers=[
        base_production_load,
        gas_injection_compressor,
        produced_water_reinjection_pump,
        seawater_injection_pump,
    ],
)

In [ ]:



model_dto = Asset(
    name="model",
    installations=[
        Installation(
            name="Installation A",
            regularity={datetime(1900, 1, 1): Expression.setup_from_expression(1.0)},
            user_defined_category=None,
            hydrocarbon_export={
                datetime(1900, 1, 1): Expression.setup_from_expression("$var.hydrocarbon_export_sm3_per_day")
            },
            fuel_consumers=[
                generator_set_a,
                flare,
            ],
            venting_emitters=[],
        )
    ],
)


In [ ]:
model_graph = model_dto.get_graph()
model = EnergyCalculator(model_graph)
consumer_results = model.evaluate_energy_usage(variables)
emission_results = model.evaluate_emissions(
    variables_map=variables,
    consumer_results=consumer_results,
)
result = GraphResult(
    graph=model_graph,
    consumer_results=consumer_results,
    variables_map=variables,
    emission_results=emission_results,
)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

print("Iterating the model components to plot results: \n")
for identity, component in model_graph.nodes.items():
    if identity in result.consumer_results:
        component_result = result.consumer_results[identity].component_result
        ds = pd.Series(component_result.energy_usage.values, index=component_result.energy_usage.timesteps)
        _ = ds.plot(xlabel="time", ylabel=component_result.energy_usage.unit, title=f"Component: {component.name} type: {type(component).__name__}", kind="line")
        plt.show(block=False)  # block=False in order to run in CI-tests.

In [ ]:
# Dummy test in order to test to assert that this notebook runs in GitHub Actions
def test_notebook_works():
    assert True